In [ ]:
import os
import sys
import cv2
import requests
import numpy as np
import skimage.io
from PIL import Image
from natsort import natsorted


## Mask-RCNN model 

use main Mask-RCNN model in mrcnn for traing.

In [ ]:
# Root directory of the project
ROOTT_DIR = os.path.abspath("PATH/to/mrcnn")

# Import Mask RCNN
sys.path.append(ROOTT_DIR)  # To find local version of the library

from mrcnn.config import Config
import mrcnn.utils
import mrcnn.model as modellib
import mrcnn.visualize
from mrcnn.model import log
from skimage.color import rgb2gray

Using TensorFlow backend.


## Configuration and loading data


In [ ]:
# Configuration for Mask R-CNN training on the teeth dataset
class TeethConfig(Config):
    NAME = "teeth"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
    NUM_CLASSES = 1 + 1  # Background + tooth
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    STEPS_PER_EPOCH = 100

config = TeethConfig()
config.display()

# Dataset class for loading images and masks
class TeethDataset(mrcnn.utils.Dataset):
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        super().__init__()

    def load_teeth(self):
        
        self.add_class("teeth", 1, "tooth") #dataset name , class id , class name
        img_folder = os.path.join(self.dataset_path, "images")
        mask_folder = os.path.join(self.dataset_path, "masks")

        # Adding each image and its corresponding mask to the dataset
        for img_name in os.listdir(img_folder):
            img_id = img_name.split(".")[0]
            img_path = os.path.join(img_folder, img_name)
            mask_path = os.path.join(mask_folder, f"{img_id}.png")

            if os.path.exists(mask_path):
                self.add_image("teeth", image_id=img_id, path=img_path, mask_path=mask_path)

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        mask_path = info['mask_path']

        # Read mask image and find contours
        mask_image = skimage.io.imread(mask_path)
        mask_image = skimage.color.rgb2gray(mask_image) if mask_image.ndim == 3 else mask_image

        ret, thresh = cv2.threshold((mask_image * 255).astype(np.uint8), 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        masks = np.zeros((mask_image.shape[0], mask_image.shape[1], len(contours)), dtype=bool)
        for i, contour in enumerate(contours):
            contour_mask = np.zeros(mask_image.shape, dtype=np.uint8)
            cv2.drawContours(contour_mask, [contour], -1, 255, thickness=cv2.FILLED)
            masks[:, :, i] = contour_mask > 127

        class_ids = np.array([1] * len(contours), dtype=np.int32)
        return masks, class_ids




Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

## Train the model

downloading weights and trining the model

In [ ]:
# Load the dataset
dataset_train = TeethDataset("PATH/to/dataset")
dataset_train.load_teeth()
dataset_train.prepare()

# Model Directory
MODEL_DIR = "PATH/to/model/directory"
COCO_MODEL_PATH = "PATH/to/coco/mask_rcnn_coco.h5" # Path to COCO trained weights

#in case you don't have trained weights uncomment the following lines
""" url = "https://github.com/ahmedfgad/Mask-RCNN-TF2/releases/download/v3.0/mask_rcnn_coco.h5"
response = requests.get(url, stream=True)

with open("mask_rcnn_coco.h5", "wb") as f:
    for chunk in response.iter_content(chunk_size=1024): 
        if chunk:
            f.write(chunk) """

# Initialize and Load the Model
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

# Which weights to start with?
init_with = "coco"  # Options: "imagenet", "coco", or "last"

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, excluding layers specific to number of classes
    model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)







Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead



ImportError: cannot import name 'topology'

In [ ]:
# Training
model.train(dataset_train, dataset_train,
            learning_rate=config.LEARNING_RATE,
            epochs=30,
            layers='heads')

""" # Fine-tuning
model.train(dataset_train, dataset_validation,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=100,
            layers="all") """
